## Calculates the Piotroski F-score for a Range of stocks

In [25]:
import requests

# replace YOUR_API_KEY with your actual API key from financialmodelingprep
api_key = '0093a06c2199659b77c42c1409f2ed53'

# replace stocks with a list of stock tickers in your portfolio
stocks = ['AAPL', 'GOOG', 'MSFT']

# initialize variables for F-Score calculation
total_points = 0
positive_points = 0

for stock in stocks:
    # retrieve financial data for the current stock
    url = f'https://financialmodelingprep.com/api/v3/financials/income-statement/{stock}?apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    
    url1 = f'https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/{stock}?apikey={api_key}'
    response1 = requests.get(url1)
    data1 = response1.json()
    
    response2 = requests.get(f"https://financialmodelingprep.com/api/v3/income-statement/{stock}?apikey={api_key}")
    data2 = response2.json()
    
    response3 = requests.get(f"https://financialmodelingprep.com/api/v3/ratios/{stock}?apikey={api_key}")
    data3 = response3.json()
    
    response4 = requests.get(f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{stock}?apikey={api_key}")
    data4 = response4.json()
    
    # calculate the relevant financial metrics
    operating_cash_flow = float(data1['financials'][0]['Operating Cash Flow'])
    # calculate the relevant financial metrics
    net_income = data2[0]["netIncome"]
    return_on_assets =  float(data3[0]["returnOnAssets"])
    accrual = operating_cash_flow / net_income - 1
    long_term_debt =  float(data3[0]["longTermDebtToCapitalization"])
    total_assets = data4[0]["totalAssets"]
    totalCurrentAssets = data4[0]["totalCurrentAssets"]
    totalCurrentLiabilities = data4[0]["totalCurrentLiabilities"]
    
    current_ratio = totalCurrentLiabilities / totalCurrentAssets
    # add points based on the F-Score criteria
    if return_on_assets > 0:
        positive_points += 1
    if accrual > 0:
        positive_points += 1
    if net_income > 0 and operating_cash_flow > 0:
        positive_points += 1
    if long_term_debt / total_assets < 0.4:
        positive_points += 1
    if current_ratio > 1:
        positive_points += 1
    
    total_points += 5  # add 5 points for each criteria met

# calculate the F-Score for the portfolio
f_score = positive_points / total_points * 10

print(f'The F-Score for the portfolio is {f_score:.2f}')


The F-Score for the portfolio is 8.67


### inventory turnover ratio

In [ ]:
import requests

# Set API key and company symbol
api_key = '0093a06c2199659b77c42c1409f2ed53'
symbol = 'AAPL'

# Retrieve revenue and COGS from the income statement for the latest period
income_url = f'https://financialmodelingprep.com/api/v3/income-statement/{symbol}?apikey={api_key}&limit=1'
income_response = requests.get(income_url).json()
revenue = float(income_response[0]['revenue'])
cogs = float(income_response[0]['costOfRevenue'])

# Retrieve inventory data from the balance sheet for the latest period
balance_url = f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{symbol}?apikey={api_key}&limit=1'
balance_response = requests.get(balance_url).json()
current_inventory = float(balance_response[0]['inventory'])
total_assets = float(balance_response[0]['totalAssets'])

# Estimate the previous inventory value as the difference between total assets and current inventory
previous_inventory = total_assets - current_inventory

# Calculate inventory turnover ratio
cost_of_goods_sold = revenue - cogs
average_inventory = (current_inventory + previous_inventory) / 2
turnover_ratio = cost_of_goods_sold / average_inventory

# Print the result
print(f'The inventory turnover ratio for {symbol} is {turnover_ratio:.2f}.')


In [49]:
import requests
import numpy as np

# Set the API endpoint and parameters
endpoint = 'https://financialmodelingprep.com/api/v3/financials/income-statement/{}'
params = {'apikey': '0093a06c2199659b77c42c1409f2ed53', 'period': 'annual'}

# Set the tickers of the stocks you want to analyze
tickers = ['AAPL', 'GOOGL', 'MSFT']

# Set the risk-free rate
risk_free_rate = 0.05

# Create a list to store the results
results = []

# Loop through the tickers and make API requests for each stock
for ticker in tickers:
    # Make the API request for the stock's income statement
    response = requests.get(endpoint.format(ticker), params=params)
    data = response.json()
    
    url1 = f'https://financialmodelingprep.com/api/v3/ratios/{ticker}?apikey=0093a06c2199659b77c42c1409f2ed53'
    response1 = requests.get(url1)
    data1 = response1.json()
    
    # Extract the annual return on equity and calculate the expected return and standard deviation
    annual_roe = [float(data1[0]['returnOnEquity']) for d in data['financials'][:5]]  # use only last 5 years' data
    expected_return = np.mean(annual_roe)
    std_dev = np.std(annual_roe)
    
    # Calculate the Sharpe ratio and store the result
    sharpe_ratio = (expected_return - risk_free_rate) / std_dev
    results.append({'Ticker': ticker, 'Expected Return': expected_return, 'Standard Deviation': std_dev, 'Sharpe Ratio': sharpe_ratio})

# Create a Pandas DataFrame to store the results
df = pd.DataFrame(results)
df.set_index('Ticker', inplace=True)

# Print the DataFrame
print(df)


<ipython-input-49-47efc648d187>:33: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (expected_return - risk_free_rate) / std_dev


        Expected Return  Standard Deviation  Sharpe Ratio
Ticker                                                   
AAPL           1.969589        0.000000e+00           inf
GOOGL          0.234134        0.000000e+00           inf
MSFT           0.436755        5.551115e-17  6.967153e+15


## Dow Jones index & Sharp Ratio for annual portfolio 

In [52]:
import yfinance as yf
import pandas as pd
import requests
from alpha_vantage.fundamentaldata import FundamentalData

# Define your API key
api_key = 'VBQK7O4DSYUTBN6S'

# Create a FundamentalData object
fd_key = FundamentalData(key=api_key, output_format='pandas')


# Define the ticker symbol for the DJIA index
ticker = "^DJI"

# Get the historical data for the DJIA index
dow_jones = yf.download(ticker, period="1y")

# Save the data to a CSV file
dow_jones.to_csv("^DJI.csv")

[*********************100%***********************]  1 of 1 completed


In [54]:
def get_net_income(income_df):
    return float(income_df.loc['netIncome'][0])

def get_roa(balance_df, income_df):
    current = float(balance_df.loc['totalAssets'][0])
    previous = float(balance_df.loc['totalAssets'][1])
    av_assets=(current+previous)/2
    return get_net_income(income_df)/av_assets

def get_ocf(cash_df):
    return float(cash_df.loc['operatingCashflow'][0])

def get_ltdebt(balance_df):
    current = float(balance_df.loc['longTermDebt'][0])
    previous = float(balance_df.loc['longTermDebt'][1])
    return previous - current

def get_current_ratio(balance_df):
    current_TCA = float(balance_df.loc['totalCurrentAssets'][0])
    previous_TCA = float(balance_df.loc['totalCurrentAssets'][1])
    current_TCL = float(balance_df.loc['totalCurrentLiabilities'][0])
    previous_TCL = float(balance_df.loc['totalCurrentLiabilities'][1])
    ratio1 = current_TCA/ current_TCL
    ratio2 = previous_TCA / previous_TCL
    return ratio1-ratio2

def get_new_shares(balance_df):
    current = float(balance_df.loc['commonStock'][0])
    previous = float(balance_df.loc['commonStock'][1])
    return current - previous 

def get_gross_margin(income_df):
    current = float(income_df.loc['grossProfit'][0])/float(income_df.loc['totalRevenue'][0])
    previous =  float(income_df.loc['grossProfit'][1])/float(income_df.loc['totalRevenue'][1])
    return current - previous

def get_asset_turnover_ratio(income_df, balance_df):
    current = float(balance_df.loc['totalAssets'][0])
    prev_1 = float(balance_df.loc['totalAssets'][1])
    prev_2 = float(balance_df.loc['totalAssets'][2])
    av_assets1=(current+prev_1)/2
    av_assets2=(prev_1+ prev_2)/2
    atr1=float(income_df.loc['totalRevenue'][0])/av_assets1
    atr2=float(income_df.loc['totalRevenue'][1])/av_assets2
    return atr1-atr2

def get_piotroski_score(income_df, balance_df, cash_df):
    score=0
    
    if get_net_income(income_df)>0:
        score +=1

    if get_roa(balance_df, income_df)>0:
        score +=1
        
    if get_ocf(cash_df)>0:
        score +=1
        
    if get_ocf(cash_df)>get_net_income(income_df):
        score +=1
        
    if get_ltdebt(balance_df)>0:
        score +=1
        
    if get_current_ratio(balance_df)>0:
        score +=1
        
    if get_new_shares(balance_df)>0:
        score +=1
        
    if get_gross_margin(income_df)>0:
        score +=1
        
    if get_asset_turnover_ratio(income_df, balance_df)>0:
        score +=1
        
    return score



def get_income_statement_annual(ticker):
    url = f'https://financialmodelingprep.com/api/v3/ratios/{ticker}?apikey=0093a06c2199659b77c42c1409f2ed53'
    response = requests.get(url)
    return response.json()

In [55]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define your portfolio of stocks with their weights
portfolio = {'AAPL': 0.6, 'AMZN': 0.3, 'MSFT': 0.1}

# Get the historical data for the portfolio stocks
tickers = list(portfolio.keys())
start_date = pd.Timestamp.now() - pd.Timedelta(days=365)
end_date = pd.Timestamp.now()
prices = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Calculate the daily returns of the portfolio
returns = prices.pct_change().dropna()

# Get the Piotroski F-score of the portfolio stocks
f_scores = {}
for ticker in portfolio.keys():
    f_scores[ticker] = get_piotroski_score(income_statement, balance_sheet, cash_flow)

# Calculate the average F-score of the portfolio
avg_f_score = np.mean(list(f_scores.values()))

# Get the Dow Jones index returns for the past year
dow_jones = pd.read_csv('^DJI.csv', index_col='Date', parse_dates=True)
dow_jones_returns = dow_jones['Close'].pct_change().dropna()
dow_jones_avg_return = dow_jones_returns.mean()

# Calculate the Sharpe ratio of the portfolio
portfolio_return = (returns * list(portfolio.values())).sum(axis=1).mean()
portfolio_volatility = returns.std().dot(list(portfolio.values()))
portfolio_sharpe_ratio = (portfolio_return - dow_jones_avg_return) / portfolio_volatility

print('Portfolio Sharpe ratio:', portfolio_sharpe_ratio)
print('Dow Jones index return:', dow_jones_avg_return)


[*********************100%***********************]  3 of 3 completed
Portfolio Sharpe ratio: -0.02417174790538655
Dow Jones index return: -7.314564531207246e-05


## how to use finpy_tse to get historical data for a specific stock:

In [ ]:
from finpy_tse import *
import pandas as pd
import finpy_tse as fpy
fpy.Get_Price_History(
    stock='خودرو',
    start_date='1400-01-01',
    end_date='1401-01-01',
    ignore_date=False,
    adjust_price=False,
    show_weekday=False,
    double_date=False)

In [ ]:
DF1 = fpy.Build_Market_StockList(bourse=True,
                                 farabourse=True,
                                 payeh=True,
                                 detailed_list=False,
                                 show_progress=True,
                                 save_excel=False,
                                 save_csv=False)

In [ ]:
print(DF1.head())

In [ ]:
Tickers = DF1.index.to_list()
Tickers